In [1]:
import os
import pandas as pd
import numpy as np

Read raw BigQuery extract into pandas dataframe

In [2]:
os.chdir('/Users/brendan/dev/onchain_data_blog/raw_data')
df = pd.read_csv("stablecoins_raw_2022-06-16.csv", low_memory=False, index_col=None, header=0)

Clean topics column and relabel

In [3]:
# parse topics column into separate columns             
df['topics'] = df['topics'].str[1:-1]
df = df.join(df['topics'].str.split(expand=True))
df = df.rename(index=str, columns={0:'topic_0', 1 :'topic_1', 2 :'topic_2'}) 
df.drop(['topics'], axis=1)

# remove quotes from topics 
l = [0,1,2]
for i in l:
    col = 'topic_' + str(i)
    df[col] = df[col].str[1:-1]

# add clean labels for stablecoin names 
df['stablecoin'] = df['address']
df['stablecoin'] = df['stablecoin'].str.replace('0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', r'USDC', regex=True)
df['stablecoin'] = df['stablecoin'].str.replace('0xdac17f958d2ee523a2206206994597c13d831ec7', r'USDT', regex=True)
df['stablecoin'] = df['stablecoin'].str.replace('0x4fabb145d64652a948d72533023f6e7a623c7c53', r'Binance USD', regex=True)
df['stablecoin'] = df['stablecoin'].str.replace('0x8e870d67f660d95d5be530380d0ec0bd388289e1', r'Pax Dollar', regex=True)
df['stablecoin'] = df['stablecoin'].str.replace('0x056fd409e1d7a124bd7017459dfea2f387b6d5cd', r'Gemini Dollar', regex=True)

#rename columns 
df = df.rename(columns={'size':'block_size','address':'contract_address'}) 

Tag activities as mints/burns, drop all else

In [4]:
# non-Gemini transactions
mints = {'0xab8530f87dc9b59234c4623bf917212bb2536d647574c8e7e5da92c2ede0c9f8':'mint', 
         '0xcb8241adb0c3fdb35b70c24ce35c5eb0c17af7431c99f827d44a445ca624176a':'mint',
         '0xf5c174d57843e57fea3c649fdde37f015ef08750759cbee88060390566a98797':'mint'}

burns = {'0xcc16f5dbb4873280815c1ee09dbd06736cffcc184412cf7a71a0fdb75d397ca5':'burn',
         '0x702d5967f45f6513a38ffc42d6ba9bf230bd40e8f53b16363c7eb4fd2deb9a44':'burn',
         '0x1b7e18241beced0d7f41fbab1ea8ed468732edbcb74ec4420151654ca71c8a63':'burn'} 

df['action'] = df['topic_0']
df.replace({'action':mints}, inplace=True)
df.replace({'action':burns}, inplace=True)

# Gemini transcations 
df['action'].mask((df['stablecoin'] == 'Gemini Dollar') & (df['action'] == "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef") & (df['topic_1'] == "0x0000000000000000000000000000000000000000000000000000000000000000"), 'mint', inplace = True)
df['action'].mask((df['stablecoin'] == 'Gemini Dollar') & (df['action'] == "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef") & (df['topic_2'] == "0x0000000000000000000000000000000000000000000000000000000000000000"), 'burn', inplace = True)

# drop other obs
df = df[df['action'].isin(['mint','burn'])]

Convert data values from hex to dec and scale for USD amounts 

In [5]:
# for non USDP/BUSD, divide by 6 decimals; for Pax Dollar and Binance Dollar divide by 18 decimals because they log raw amounts 
df = df[df.data != "0x"] 
df['data'] = df['data'].apply(int, base=16)

def log_amount_to_USD(contract_address: str, amount: int):
    if contract_address == "0x8e870d67f660d95d5be530380d0ec0bd388289e1":
        return amount / 1e18
    elif contract_address == "0x4fabb145d64652a948d72533023f6e7a623c7c53":
        return amount / 1e18
    elif contract_address == "0x056fd409e1d7a124bd7017459dfea2f387b6d5cd": 
                return amount / 1e2
    else:
        return amount / 1e6

df['amount_usd'] = np.vectorize(log_amount_to_USD, otypes=[float])(df['contract_address'], df['data'])

Preview data frame

In [6]:
df.head()

,log_index,transaction_hash,transaction_index,contract_address,data,topics,block_timestamp,block_number,block_hash,number,...,block_size,gas_limit,gas_used,base_fee_per_gas,topic_0,topic_1,topic_2,stablecoin,action,amount_usd
196,237,0x6a71395ef84c74a3ea6256b4384f99d30b343f87d004...,306,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,528080000,'0xcc16f5dbb4873280815c1ee09dbd06736cffcc18441...,2022-06-16 19:17:18+00:00,14975009,0x883fdaff75e30b7a75f0d821a9f8f481e367427f7871...,14975009,...,154602,30029295,30023404,25109681000,0xcc16f5dbb4873280815c1ee09dbd06736cffcc184412...,0x00000000000000000000000055fe002aeff02f77364d...,None,USDC,burn,528.08
813,108,0xc0284db08042e2cbb5c28020f771def0dd1f992d0729...,91,0x4fabb145d64652a948d72533023f6e7a623c7c53,97384865830000000000000000,'0xf5c174d57843e57fea3c649fdde37f015ef08750759...,2022-06-16 16:35:33+00:00,14974347,0x6f0ad04a07bdacf3b402a2879e663682457bb2c98223...,14974347,...,119367,29970705,29534478,57120871265,0xf5c174d57843e57fea3c649fdde37f015ef08750759c...,0x000000000000000000000000e25a329d385f77df5d4e...,None,Binance USD,mint,97384865.83
868,106,0xa85d7ac253df50a3d5ea6e1339681705acf765bc2507...,90,0x8e870d67f660d95d5be530380d0ec0bd388289e1,209559120000000000000000,'0xf5c174d57843e57fea3c649fdde37f015ef08750759...,2022-06-16 16:35:33+00:00,14974347,0x6f0ad04a07bdacf3b402a2879e663682457bb2c98223...,14974347,...,119367,29970705,29534478,57120871265,0xf5c174d57843e57fea3c649fdde37f015ef08750759c...,0x000000000000000000000000e25a329d385f77df5d4e...,None,Pax Dollar,mint,209559.12
931,104,0x8a5cb538fbdc52029855b4e3d8018a6bda64266feb45...,51,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,71709696380000,'0xab8530f87dc9b59234c4623bf917212bb2536d64757...,2022-06-16 15:57:30+00:00,14974191,0xc51f2635c13aac11d9b078bb066cf41949042c03a606...,14974191,...,122008,30000000,28998185,40973823338,0xab8530f87dc9b59234c4623bf917212bb2536d647574...,0x0000000000000000000000005b6122c109b78c675548...,0x00000000000000000000000055fe002aeff02f77364d...,USDC,mint,71709696.38
962,402,0x4951280d7bd31396e366c63fbfb01ff6e875d7ad1739...,138,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,142680000,'0xcc16f5dbb4873280815c1ee09dbd06736cffcc18441...,2022-06-16 18:50:58+00:00,14974897,0x193153d5bd6a01d02940dd08e8b191e4bd89c7fdd7ec...,14974897,...,118664,29970677,24197655,35869026529,0xcc16f5dbb4873280815c1ee09dbd06736cffcc184412...,0x00000000000000000000000055fe002aeff02f77364d...,None,USDC,burn,142.68
